In [1]:
import pycolmap 
from pathlib import Path 
import open3d as o3d 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
scene_folder = Path('../data/small_city_road_vertical')
images_folder = scene_folder / 'images'

database_path = scene_folder / 'database.db'

### Sparse Reconstruction

In [ ]:
pycolmap.extract_features(database_path, images_folder)
pycolmap.match_exhaustive(database_path)
maps = pycolmap.incremental_mapping(database_path, images_folder, scene_folder)
maps[0].write(scene_folder)

Convert to PLY format, suitable for 3D data. (Sparse point cloud)

In [10]:
reconstruction = pycolmap.Reconstruction(scene_folder)
reconstruction.export_PLY(scene_folder / 'sparse.ply')

In [13]:
cloud = o3d.io.read_point_cloud(str(scene_folder / 'sparse.ply'))
o3d.visualization.draw_geometries([cloud])  

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
